<br></br>
# **자연어와 Deep Learning**
## **LSTM 단어 알파벳 완성모델**

<br></br>
## **1 분석 Data**
1. X (Input Data) : Sequence Data (전체 4개 알파벳의 연속성을 학습)
1. Y (Output Data) : Sequence Data (전체 2개 한글의 연속성을 학습)
1. 일반 회기식 에서는 Output 이 1개만 가능하
1. Sequence Data 를 Output으로 연결하기 위해 LSTM Cell을 활용

In [1]:
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], 
            ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

<br></br>
## **2 데이터 임배딩 객체/ 함수(Batch 함수) 정의**
one-hot-encoding 활용

In [2]:
char_arr = [ c for c in 'SPabcdefghijklmnopqrstuvwxyz나놀녀단랑무사소스어이키E'] 
num_dic  = { n : i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

In [3]:
# Tensorflow 에 입력을 위한 Batch 함수를 정의한다
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])] 
        target = [num_dic[n] for n in (seq[1] + 'E')]
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    return input_batch, output_batch, target_batch

<br></br>
## **2 모델의 정의**

In [4]:
import tensorflow as tf
import numpy as np

In [5]:
tf.reset_default_graph()
learning_rate = 0.01
n_hidden, total_epoch = 128, 100
n_class = n_input = dic_len

enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets   = tf.placeholder(tf.int64, [None, None])  # [batch size, time steps]

In [6]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob = 0.5) 
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,dtype=tf.float32)

In [7]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, 
                                            initial_state = enc_states, 
                                            dtype = tf.float32)

In [8]:
model = tf.layers.dense(outputs, n_class, activation=None)
cost  = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits = model, labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

<br></br>
## **3 모델의 학습**

In [9]:
%%time
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 9 == 0 :
        print('Epoch: {:4d}  cost = {:.6f}'.format((epoch + 1),loss))
print('최적화 완료!')

Epoch:    1  cost = 3.790564
Epoch:   10  cost = 0.198150
Epoch:   19  cost = 0.124658
Epoch:   28  cost = 0.006218
Epoch:   37  cost = 0.002358
Epoch:   46  cost = 0.002709
Epoch:   55  cost = 0.000967
Epoch:   64  cost = 0.000666
Epoch:   73  cost = 0.003260
Epoch:   82  cost = 0.001237
Epoch:   91  cost = 0.000878
Epoch:  100  cost = 0.000768
최적화 완료!
CPU times: user 1.56 s, sys: 373 ms, total: 1.94 s
Wall time: 1.53 s


<br></br>
## **4 모델의 검증**

In [10]:
%%time
def translate(word):
    seq_data   = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)   # [None, None, n_input]
    result     = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    decoded    = [char_arr[i] for i in result[0]]
    end        = decoded.index('E')
    translated = ''.join(decoded[:end])
    return translated

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs


In [11]:
print('\n=== 번역 테스트 ===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))
sess.close()


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 놀이
